In [1]:
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer 
import os
from nltk.stem.porter import PorterStemmer

In [2]:
docs = []
book_names = []

for book in os.listdir('./DataBooks'):
    book_names.append(book)
    f = open("Databooks/" +book, encoding='utf-8')
    raw = f.read()
    #raw = raw[raw.find("*** START OF THIS PROJECT GUTENBERG EBOOK"):raw.find("*** END OF THIS PROJECT GUTENBERG EBOOK"):]
    raw = raw[raw.find("START OF"):raw.find("END OF"):]
    raw_lowerT = raw.lower()
    raw_lower = re.sub('[^a-z]', ' ', raw_lowerT)
    translator = str.maketrans('', '', string.punctuation)
    raw_punc = raw_lower.translate(translator)
    raw_space = " ".join(raw_punc.split()) 
    stop_words = set(stopwords.words("english"))
    sw = ['time', 'always', 'things', 'seem', 'things', 'call' ,'even', 'every', 'upon', 'little', 'theres', 'thats', 'illustration', 'want', 'didnt', 'couldnt', 'never', 'project', 'gutenberg', 'ebook', 'good', 'thing', 'mean', 'cant', 'great', 'well', 'might', 'much', 'many', 'come', 'could', 'know', 'make', 'think', 'tell', 'could', 'find', 'aint', 'dont', 'give', 'look', 'take', 'must']
    for word in sw:
        stop_words.add(word)
    #stemmer = PorterStemmer()
    word_tokens = word_tokenize(raw_space)
    #stems = [stemmer.stem(word) for word in word_tokens]
    filtered_text = [word for word in word_tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmasT = [lemmatizer.lemmatize(word, pos ='v') for word in filtered_text]
    lemmasV = [word for word in lemmasT if len(word)>3]
    lemmas = [word for word in lemmasV if word not in stop_words]
    docs.append(' '.join(lemmas))
    #stopwords = STOPWORDS
    wc = WordCloud(background_color='white',
                #stopwords=stopwords,
                height=400,
                width=400,
                max_words=20,
                min_word_length=4)
    wordcloud = wc.generate(' '.join(lemmas))
    wordcloud.to_file("wordcloud/" +book[:-4] +".png")

In [3]:
# settings that you use for count vectorizer will go here 
tfidf_vectorizer = TfidfVectorizer(use_idf=True) 
 
# just send in all your docs here 
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)

tf_idf_matrix = pd.DataFrame(index = tfidf_vectorizer.get_feature_names())
i = 0
for vector in tfidf_vectorizer_vectors:
    tf_idf_matrix.insert(i , book_names[i], vector.T.todense(), allow_duplicates=False)
    i += 1
#tf_idf_matrix.sort_values(by=['alice-in-wonderland.txt'], inplace=True, ascending=False)
tf_idf_matrix

,alice-in-wonderland.txt,Among the Forest People.txt,An Introductory Course of Quantitative Chemical Analysis.txt,Curious Myths of the Middle Ages.txt,"Democracy In America, Volume 1 (of 2).txt",Experiments and Observations on Different.txt,"Formation of the Union, 1750-1829.txt",Histories of two hundred and fifty-one divisions of the German army which participated in the wa.txt,History of King Charles The First of England.txt,History of Phosphorus.txt,...,The Tale of Timmy Tiptoes.txt,The Threefold Commonwealth.txt,The Toxicity of Caffein An experimental study on different species of animals.txt,The United States of America Part I.txt,The White Feather.txt,Three Minute Stories.txt,Through the Looking-Glass.txt,Tiger and Tom and Other Stories for Boys.txt,Tom Sawyer Abroad.txt,War and Peace.txt
aachen,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000657,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
aadites,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
aarhorn,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
aaron,0.0,0.0,0.0,0.00648,0.0,0.0,0.0,0.000000,0.0,0.010564,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009943,0.0,0.0
aaschik,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zygomalas,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
zygomatic,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
zymase,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.024440,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
zymotechnia,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [28]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random

#X = sparse_random(100, 100, density=0.01, format='csr', random_state=42)

svd = TruncatedSVD(n_components=3)

svd.fit_transform(tf_idf_matrix)
#TruncatedSVD(n_components=3)

#print(svd.explained_variance_ratio_)
#print(svd.explained_variance_ratio_.sum())
#print(svd.singular_values_)

topic1 = svd.components_[0]
topic2 = svd.components_[1]
topic3 = svd.components_[2]

lsa = pd.DataFrame(list(zip(book_names, topic1, topic2, topic3)), columns=['book', 'topic1', 'topic2', 'topic3'])
lsa.sort_values(by=['topic1'], inplace=True, ascending=False)                                    
lsa



,book,topic1,topic2,topic3
31,The Last Leaf.txt,0.229572,-0.009809,-0.089324
49,Tiger and Tom and Other Stories for Boys.txt,0.207382,-0.128170,0.054064
3,Curious Myths of the Middle Ages.txt,0.206713,0.055289,-0.092677
12,Medieval People.txt,0.206195,-0.002879,-0.081702
15,O Pioneers.txt,0.201412,-0.137940,0.073004
29,The History of England from the Accession of J...,0.198242,0.084447,-0.229723
25,The French Revolution.txt,0.197118,0.007223,-0.118720
47,Three Minute Stories.txt,0.192844,-0.183784,0.134394
50,Tom Sawyer Abroad.txt,0.188963,-0.079022,0.036806
18,Sandman_s Goodnight Stories.txt,0.186661,-0.181327,0.140906


In [51]:
topic_1 = []
topic_2 = []
topic_3 = []

for index, row in lsa.iterrows():
    if row['topic1'] > row['topic2'] and row['topic1'] > row['topic3']:
        topic_1.append(row['book'])
    if row['topic2'] > row['topic1'] and row['topic2'] > row['topic3']:
        topic_2.append(row['book'])
    if row['topic3'] > row['topic2'] and row['topic3'] > row['topic1']:
        topic_3.append(row['book'])
        
print("topic 1: " + str(topic_1) + "\n")
print("topic 2: " + str(topic_2) + "\n")
print("topic 3: " + str(topic_3) + "\n")

topic 1: ['The Last Leaf.txt', 'Tiger and Tom and Other Stories for Boys.txt', 'Curious Myths of the Middle Ages.txt', 'Medieval People.txt', 'O Pioneers.txt', 'The History of England from the Accession of James II.txt', 'The French Revolution.txt', 'Three Minute Stories.txt', 'Tom Sawyer Abroad.txt', 'Sandman_s Goodnight Stories.txt', 'Mother Storie.txt', 'The Magic of Oz.txt', 'The Ruins.txt', 'How the Flag Became Old Glory.txt', 'Among the Forest People.txt', 'The Princess and the Goblin.txt', 'The Rose and the Ring.txt', 'The Greater Republic.txt', 'War and Peace.txt', 'The History Of The Decline And Fall Of The Roman Empire.txt', 'History of King Charles The First of England.txt', 'Little Lord Fauntleroy.txt', 'Prince Prigio.txt', 'The Railway Children.txt', 'The Magic Fishbone.txt', 'The Secret Garden.txt', 'Peter-pan.txt', 'The Eighteenth Brumaire of Louis Bonaparte.txt', 'Through the Looking-Glass.txt', 'The White Feather.txt', 'alice-in-wonderland.txt']

topic 2: ['The Progres

In [22]:
n_features = 1000
n_components = 3
n_top_words = 10
lsa_topics=['topic1', 'topic2', 'topic3']

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english', ngram_range=(1, 2), use_idf = True)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)

tf_idf_matrix = pd.DataFrame(index = tfidf_vectorizer.get_feature_names())
i = 0
for vector in tfidf_vectorizer_vectors:
    tf_idf_matrix.insert(i , book_names[i], vector.T.todense(), allow_duplicates=False)
    i += 1
#tf_idf_matrix.sort_values(by=['alice-in-wonderland.txt'], inplace=True, ascending=False)
tf_idf_matrix

,alice-in-wonderland.txt,Among the Forest People.txt,An Introductory Course of Quantitative Chemical Analysis.txt,Curious Myths of the Middle Ages.txt,"Democracy In America, Volume 1 (of 2).txt",Experiments and Observations on Different.txt,"Formation of the Union, 1750-1829.txt",Histories of two hundred and fifty-one divisions of the German army which participated in the wa.txt,History of King Charles The First of England.txt,History of Phosphorus.txt,...,The Tale of Timmy Tiptoes.txt,The Threefold Commonwealth.txt,The Toxicity of Caffein An experimental study on different species of animals.txt,The United States of America Part I.txt,The White Feather.txt,Three Minute Stories.txt,Through the Looking-Glass.txt,Tiger and Tom and Other Stories for Boys.txt,Tom Sawyer Abroad.txt,War and Peace.txt
abandon,0.000000,0.000000,0.000000,0.005832,0.022374,0.000000,0.0,0.000604,0.007453,0.005926,...,0.000000,0.000000,0.000000,0.000000,0.001660,0.000000,0.000000,0.000000,0.000000,0.015574
able,0.003115,0.005092,0.000477,0.026394,0.022501,0.027993,0.0,0.000608,0.006558,0.000000,...,0.000000,0.034039,0.004753,0.000000,0.006262,0.017215,0.005376,0.014072,0.000000,0.011858
accept,0.000000,0.000000,0.000000,0.005362,0.003047,0.000000,0.0,0.000000,0.003426,0.000000,...,0.000000,0.006383,0.000579,0.000000,0.003053,0.000000,0.000000,0.014293,0.000000,0.008375
accord,0.000000,0.000000,0.004641,0.068935,0.009119,0.009508,0.0,0.002708,0.015188,0.028984,...,0.000000,0.016036,0.016437,0.000000,0.002030,0.000000,0.000000,0.005069,0.014930,0.011378
account,0.002998,0.000000,0.009643,0.086787,0.006918,0.040795,0.0,0.001754,0.030656,0.000000,...,0.000000,0.005040,0.005489,0.000000,0.002411,0.005523,0.000862,0.020314,0.033237,0.009919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yield,0.000000,0.000000,0.010468,0.008043,0.006095,0.020473,0.0,0.000093,0.020555,0.000000,...,0.000000,0.003192,0.000000,0.000000,0.000000,0.000000,0.000000,0.022870,0.000000,0.003647
york,0.001240,0.006081,0.000000,0.002627,0.017914,0.000955,0.0,0.000000,0.022377,0.021351,...,0.052665,0.001042,0.000000,0.073783,0.000000,0.000000,0.000000,0.022406,0.032994,0.000132
young,0.003998,0.191115,0.000000,0.033868,0.003910,0.006928,0.0,0.002631,0.020738,0.004302,...,0.000000,0.000840,0.017839,0.000000,0.010245,0.049704,0.004311,0.142196,0.006647,0.047461
youth,0.000000,0.000000,0.000000,0.017496,0.001657,0.001060,0.0,0.000000,0.004968,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010793,0.000000,0.000000,0.037312,0.000000,0.006465


In [26]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random

lsa = TruncatedSVD(n_components = n_components, random_state = 1, algorithm = 'arpack')

lsa.fit(tfidf_vectorizer_vectors)

def get_lsa_topics(lsa, vectorizer, topics, n_top_words=n_top_words):
    word_dict = {}
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(lsa.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        word_dict[topics[topic_idx]] = top_features

    return pd.DataFrame(word_dict)

get_lsa_topics(lsa, tfidf_vectorizer, lsa_topics)

,topic1,topic2,topic3
0,like,acid,acid
1,king,water,water
2,long,state,solution
3,mother,form,mother
4,people,solution,heat
5,hear,experiment,tube
6,hand,unite state,soap
7,water,contain,peter
8,away,unite,experiment
9,house,heat,temperature


In [55]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tf_idf_matrix)
cosine_sim

array([[1.        , 0.43463303, 0.55344617, ..., 0.18984752, 0.3301713 ,
        0.59939284],
       [0.43463303, 1.        , 0.35663241, ..., 0.42858276, 0.45933342,
        0.34075929],
       [0.55344617, 0.35663241, 1.        , ..., 0.30707519, 0.40869502,
        0.38658229],
       ...,
       [0.18984752, 0.42858276, 0.30707519, ..., 1.        , 0.60602317,
        0.15498233],
       [0.3301713 , 0.45933342, 0.40869502, ..., 0.60602317, 1.        ,
        0.39563062],
       [0.59939284, 0.34075929, 0.38658229, ..., 0.15498233, 0.39563062,
        1.        ]])